<http://qiita.com/ynakayama/items/6a472e5ebbe9365186bd>

# 決定木で未来の株価を予測する

## 概略
1. 教師データの数値の配列 (train_X) と結果の配列 (train_y) を学習させる
2. テストデータの数値の配列 (test_X) を与えると予測結果 (test_y) が帰ってくる

## 使い方

1. 教師データをつくる
    * train_X (教師データの配列) と train_y (それに対する 1 か 0 かのラベル) が返ってくる
2. リターンインデックスを算出する
3. リターンインデックスの変化を決定木に学習させる
4. リターンインデックスから教師データを抽出し分類器に学習させます。

----

# ニコン株価データ取得　
* http://stocks.finance.yahoo.co.jp/stocks/detail/?code=7731.T

In [ ]:
# 第一引数に銘柄コード
# 第二引数に日付 (2015-01-01 など) 
# ニコン 7731
# 日立ハイテクノロジー 8036
# ディスコ 6146
# スクリーン 7735
# アドテス 6857
#%run JpStock.py 7731 "2000-01-01"
#%run JpStock.py 8036 "2000-01-01"
#%run JpStock.py 6146 "2000-01-01"
%run JpStock.py 7735 "2000-01-01"
%run JpStock.py 6857 "2000-01-01"

# 株価を予想する
    * 10/13 までの 90 営業日のデータをもとに予測するとこうなりました。
    * 答えは 0 、つまり株価は下がることになります。

In [3]:
# arg1  銘柄コード (ニコン7731 )
# arg2 学習させる期間開始日　(2016-01-01 など) 
# arg2 予測させる日時 (2016-01-01 など) 
# arg3 学習させる日数　（例 15 : 15日間) 
# arg4 学習アルゴリズム番号 0: 決定木
%run 20161113_PredictStock.py 7731 "2000-01-06" "2016-11-18" "Lrn"
#%run 20161113_PredictStock.py 7731 "2000-01-06" "2016-11-18"  "Predict" 30

From 2006-01-06 to 2016-11-09 by 30
(1/204) False step(30) 2008-4-1 [Predict : Down] Result: ¥80
(2/204) True step(30) 2008-5-1 [Predict : Down] Result: ¥0
...Retry maybe weekend day
...Retry maybe weekend day
(3/204) True step(30) 2008-6-2 [Predict : Down] Result: ¥-20
(4/204) False step(30) 2008-7-1 [Predict : Down] Result: ¥10
(5/204) True step(30) 2008-8-1 [Predict : Down] Result: ¥-150
...Retry maybe weekend day
...Retry maybe weekend day
(6/204) True step(30) 2008-9-1 [Predict : Down] Result: ¥-60
(7/204) True step(30) 2008-10-1 [Predict : Down] Result: ¥-85
...Retry maybe weekend day
...Retry maybe weekend day
... Skip calc because of NAN
...Retry maybe weekend day
...Retry maybe weekend day
(9/204) True step(30) 2008-12-1 [Predict : Down] Result: ¥-49
... Skip calc because of NAN
...Retry maybe weekend day
...Retry maybe weekend day
(11/204) True step(30) 2009-2-2 [Predict : Down] Result: ¥-45
...Retry maybe weekend day
...Retry maybe weekend day
(12/204) True step(30) 2009-3-2

KeyboardInterrupt: 

---

# 解説

## 教師データの作成
* まずは一番面倒な株価の調整後終値から教師データを作るまでのコードを用意します。
* これは終値のリストを渡すと train_X と train_y が返るようにすれば良いでしょう。
###  リターンインデックスを算出する
    * 生の株価データそのままですと、会社ごとに価格帯も全然ちがいますから教師データとしてはちょっと使いづらい
    * 正規化しても良いのですが、ここは資産価値の変化をあらわすリターンインデックスに注目しましょう。
    * 算出方法は前回も書きましたがこのように pandas で求まります。
    * 金融の世界におけるリターンとは通常はある日を起算日とした資産価格のパーセント変化を指します。
    * 単純なリターンインデックスは pandas を利用して次のように求まります。

In [1]:
import pandas as pd
import numpy as np
def get_ret_index(close):
    # リターンインデックス データーが昇順（日付が過去が上になって最新が一番下）になっている前提
    returns = pd.Series(close).pct_change() # 騰落率を求める
    ret_index = (1 + returns).cumprod() # 累積積を求める
    ret_index.iloc[0] = 1 # 最初の値を 1.0 にする
    return ret_index

def learn_data(arr,step):
    #train_X (教師データの配列) と train_y (それに対する 1 か 0 かのラベル) が戻る
    learn_X = []; 
    learn_y = [];
    flg=0;
    for i in np.arange(0, len(arr)-step):
        end = i + step# step日間の変化を素性にする
        data = arr.iloc[i:end]
        close= data['AdjClose']
        close.head()
        feature = get_ret_index(close)
        # その翌日、株価は上がったか？
        if close.iloc[-1] < arr['AdjClose'].iloc[end]:
            flg=1
        else:
            flg=0
        #print(close.iloc[-1],"<",arr['AdjClose'].iloc[end],flg)
        learn_X.append(feature.values)
        learn_y.append(flg) # YES なら 1  NO なら 0 
    # 上げ下げの結果と教師データのセットを返す
    return np.array(learn_X), np.array(learn_y)

csvfile="stock_7731.csv"
df = pd.read_csv(csvfile, index_col=0, parse_dates=True)
df = df.dropna()
#df.head()
step = 15
LrnX,LrnY =learn_data(df,step)

## リターンインデックスの変化を決定木に学習させる
    * さてここからがキモ。こうして求まったリターンインデックスから教師データを抽出し分類器に学習させます

In [40]:
from sklearn.tree import DecisionTreeClassifier
# リターンインデックスを教師データを取り出す
# 決定木のインスタンスを生成
clf = DecisionTreeClassifier()
# 学習させる
clf.fit(LrnX, LrnY)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

## うまく学習したかどうか分類器を試す
    * まずはテストとして、教師データとまったく同じデータをテストデータとして流してみます。

In [93]:
test_y = []
for i in np.arange(0,len(LrnX)-step):
    end = i + step
    # リターンインデックスと同じ期間をテストとして分類させてみる
    test_x = LrnX[i:end]
    #print(type(test_x))
    # 結果を格納して返す
    result = clf.predict(test_x)
    test_y.append(result[0])

print(LrnY) # 期待すべき答え
#=> [1 1 1 0 1 1 0 0 0 1 0 1 0 0 0]
print(np.array(test_y)) # 分類器が出した予測
#=> [1 1 1 0 1 1 0 0 0 1 0 1 0 0 0]

[1 1 1 0 0 0 1]
[]


# 　学習方法を工夫する
## 交差検証法
1. cross_val_score
* 今回やるのは学習用データーを何分割かして（例えば５分割）、ひとつをテスト用データーにつかい残りを教師用データーとしてもちいながら、
* テスト用データーを変えながら5回繰り返す、というやり方です。
* 図で説明するとわかりやすいんですが、わざわざ説明図を書くのも面倒なので、もうそのままソースコードのせます。
* ライブラリにそのものの機能が提供されているのでそれを使えば実際に使いながら試せたりします。

In [29]:
from sklearn.model_selection import cross_val_score
# meanで学習結果の正解率まで求められる。
scores = cross_val_score(clf, LrnX, LrnY, cv=5)
mean = scores.mean()
print(mean)

0.503340080972


## 予測の妥当性
* 学習量を変えて総当たりで結果を求めて、一番正解率が高かった学習量を採用すればいいのではないか

In [ ]:
def get_params(ccode, last_date, db):
    '''
    決定木に最適なパラメーターを総当たりで取得します。
    '''
    _top_mean = 0
    _data_length = 0
    _step = 0
    _min_samples_leaf = 0
    _max_depth = 0
    for data_length in range(30, 40, 10):
        for step in range(4, 12):
            for min_samples_leaf in range(1, 7):
                for max_depth in range(2, 8):
                    clf, LrnX, LrnY = lean(ccode, last_date, db, data_length, step, min_samples_leaf, max_depth)
                    scores = cross_val_score(clf, LrnX, LrnY, cv=5)
                    mean = scores.mean()
                    del clf
                    if _top_mean < mean:
                        _top_mean = mean
                        _data_length = data_length
                        _step = step
                        _min_samples_leaf = min_samples_leaf
                        _max_depth = max_depth
#     print("{}%, length:{}, step:{}, leaf: {}, depth: {}".format(int(_top_mean * 100), _data_length, _step, _min_samples_leaf, _max_depth))
    return _top_mean, _data_length, _step, _min_samples_leaf, _max_depth
